In [26]:
import pandas as pd
from urllib.request import urlopen
from urllib.error import HTTPError
import json 
import numpy as np

In [22]:
fname = '6.1331_TW_others_Zheng.xlsx'
df = pd.read_excel(fname)
df

,PY,JI,TI,AU,AB,C1,AU_CO,AU1_CO,AUc_CO,N_AU,...,DI,DT,ID,PU,SO,TC,VL,DB,AU_UN,AUc_UN
0,1978.0,AAPG Bull.-Am. Assoc. Petr. Geol.,PLATE CONVERGENCE AND ACCRETION IN TAIWANLUZON...,BOWIN C;LU R;CHAOSHING L;SCHOUTEN H,NaN,"NATL TAIWAN UNIV,TAIPEI,TAIWAN.;",TAIWAN,TAIWAN,USA,4,...,NaN,Article,NaN,AMER ASSOC PETROLEUM GEOLOGIST,AAPG BULLETIN-AMERICAN ASSOCIATION OF PETROLEU...,182,62,ISI,NATL TAIWAN UNIV,WOODS HOLE OCEANOG INST
1,2011.0,AAPG Mem.,Two contrasting kinematic styles of active fol...,Yue L;Suppe J;Hung J,Two adjacent active thrust ramps in western Ta...,"Department of Geosciences, Princeton Universit...",USA;TAIWAN;TAIWAN;TAIWAN,USA,USA,3,...,10.1306/13251337M943431,Article,coseismic process; deformation; detachment f...,NaN,AAPG Memoir,18,NaN,scopus,PRINCETON UNIVERSITY;NATIONAL TAIWAN UNIVERSIT...,DEPARTMENT OF GEOSCIENCES
2,1977.0,Agricultural Administration,ChineseAmerican Joint Commission on Rural Reco...,Wang Y,The Chinese-American Joint Commission on Rural...,"Secretary General, Joint Commission on Rural R...",TAIWAN,TAIWAN,TAIWAN,1,...,10.1016/0309-586X(77)90003-6,Article,NaN,NaN,Agricultural Administration,1,4,scopus,NaN,NOTREPORTED
3,1991.0,AI Applications in Natural Resource Management,Expert systems for national forest management ...,Yong-Chi Y Y;Hui-Yi Y H,"The national forests of Taiwan, which encompas...","Dept. of Forestry, National Taiwan Univ., Taip...",TAIWAN;TAIWAN;TAIWAN,TAIWAN,NaN,2,...,NaN,Article,afforestation; developing country; expert sy...,NaN,AI Applications in Natural Resource Management,2,5,scopus,NaN,NaN
4,2010.0,Am. J. Sci.,ACCRETIONARY OROGEN AND EVOLUTION OF THE JAPAN...,Jahn B,The Japanese Islands represent a segment of a ...,"Acad Sinica, Inst Earth Sci, Taipei 11529, Tai...",TAIWAN,TAIWAN,TAIWAN,1,...,10.2475/10.2010.02,Article,A-TYPE GRANITES; ARABIAN-NUBIAN SHIELD; PALEOZ...,AMER JOURNAL SCIENCE,AMERICAN JOURNAL OF SCIENCE,72,310,ISI,INST EARTH SCI,NATL TAIWAN UNIV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,2001.0,NaN,Ionospheric foF2 variations prior to strong ea...,Chuo Y;Chen Y;Liu J;Pulinets S,Many studies of the seismo-ionospheric couplin...,"Natl Cent Univ, Inst Space Sci, Chungli 320, T...",TAIWAN;TAIWAN;TAIWAN;RUSSIA,TAIWAN,TAIWAN,4,...,10.1016/S0273-1177(01)00209-5,Article; Proceedings Paper,CANNOT EARTHQUAKES; LIGHTS; PREDICTION,ELSEVIER SCIENCE BV,ADVANCES IN REMOTE SENSING OF THE MIDDLE AND U...,19,27,ISI,NATL CENT UNIV;NATL CENT UNIV;NATL CENT UNIV;I...,NATL CENT UNIV
1327,2001.0,NaN,Methodology for improving weak foundations by ...,Chang D;Chang J;Chang J,Leader Administration College planned to build...,"Natl Chung Yuan Univ, Dept Civil Engn, Chungli...",TAIWAN;TAIWAN,TAIWAN,TAIWAN,3,...,NaN,Article; Proceedings Paper,NaN,TRANSPORTATION RESEARCH BOARD NATL RESEARCH CO...,"GEOMATERIALS 2001: SOILS, GEOLOGY, AND FOUNDAT...",0,NaN,ISI,NATL CHUNG YUAN UNIV,NATL CHUNG YUAN UNIV
1328,2003.0,NaN,Strength properties of flexible geogrids under...,Hsieh C;Lin C,The outdoor ultraviolet (UV) light exposure pr...,"Natl Pingtung Univ Sci & Technol, Dept Civil E...",TAIWAN,TAIWAN,TAIWAN,2,...,NaN,Article; Proceedings Paper,GEOTEXTILES,TRANSPORTATION RESEARCH BOARD NATL RESEARCH CO...,"SOIL MECHANICS 2003: SOILS, GEOLOGY, AND FOUND...",2,NaN,ISI,NATL PINGTUNG UNIV SCI AND TECHNOL,NATL PINGTUNG UNIV SCI AND TECHNOL
1329,2004.0,NaN,Taiwans cooperative space activities at presen...,Ip W,Taiwan is developing a long-term space program...,"Natl Cent Univ, Inst Astron & Space Sci, Chung...",TAIWAN,TAIWAN,TAIWAN,1,...,10.1016/j.asr.2003.04.055,Article; Proceedings Paper,NaN,PERGAMON-ELSEVIER SCIENCE LTD,SPACE SCIENCE EDUCATION AND PROMOTING NORTH-SO...,0,34,ISI,NATL CENT UNIV,NATL CENT UNIV


In [23]:
df = df.dropna(subset=['DI'])
df

,PY,JI,TI,AU,AB,C1,AU_CO,AU1_CO,AUc_CO,N_AU,...,DI,DT,ID,PU,SO,TC,VL,DB,AU_UN,AUc_UN
1,2011.0,AAPG Mem.,Two contrasting kinematic styles of active fol...,Yue L;Suppe J;Hung J,Two adjacent active thrust ramps in western Ta...,"Department of Geosciences, Princeton Universit...",USA;TAIWAN;TAIWAN;TAIWAN,USA,USA,3,...,10.1306/13251337M943431,Article,coseismic process; deformation; detachment f...,NaN,AAPG Memoir,18,NaN,scopus,PRINCETON UNIVERSITY;NATIONAL TAIWAN UNIVERSIT...,DEPARTMENT OF GEOSCIENCES
2,1977.0,Agricultural Administration,ChineseAmerican Joint Commission on Rural Reco...,Wang Y,The Chinese-American Joint Commission on Rural...,"Secretary General, Joint Commission on Rural R...",TAIWAN,TAIWAN,TAIWAN,1,...,10.1016/0309-586X(77)90003-6,Article,NaN,NaN,Agricultural Administration,1,4,scopus,NaN,NOTREPORTED
4,2010.0,Am. J. Sci.,ACCRETIONARY OROGEN AND EVOLUTION OF THE JAPAN...,Jahn B,The Japanese Islands represent a segment of a ...,"Acad Sinica, Inst Earth Sci, Taipei 11529, Tai...",TAIWAN,TAIWAN,TAIWAN,1,...,10.2475/10.2010.02,Article,A-TYPE GRANITES; ARABIAN-NUBIAN SHIELD; PALEOZ...,AMER JOURNAL SCIENCE,AMERICAN JOURNAL OF SCIENCE,72,310,ISI,INST EARTH SCI,NATL TAIWAN UNIV
5,2021.0,Am. J. Sci.,American Journal of Science,Chang Q;Hren M;Lin A;Tabor C;Yu S;Eley Y;Harris G,Fluvial sediments are important archives of pa...,"Univ Connecticut, Dept Geosci, Storrs, CT 0626...",USA;TAIWAN;TAIWAN;UNITED KINGDOM,USA,USA,7,...,10.2475/04.2021.01,Article,CARBON-ISOTOPE DISCRIMINATION; LAST GLACIAL MA...,AMER JOURNAL SCIENCE,AMERICAN JOURNAL OF SCIENCE,1,321,ISI,UNIV CONNECTICUT;NATL CENT UNIV;EXPLORAT AND D...,UNIV CONNECTICUT
6,2006.0,Anatolia,Assessing a river tracing behavioural model A ...,Lee T,NaN,Graduate School of Leisure and Exercise Studie...,TAIWAN,TAIWAN,TAIWAN,1,...,10.1080/13032917.2006.9687194,Article,model; river; tourism; tourist behavior; t...,Anatolia,Anatolia,9,17,scopus,NATIONAL YUNLIN UNIVERSITY OF SCIENCE AND TECH...,電子郵件: THLEE@YUNTECH.EDU.TW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321,1988.0,NaN,MICROSTRUCTURE OF CHROMIUM SPINEL AND PYRITE A...,HWANG S;SHEN P;CHU H;JENG R,NaN,"CENT GEOL SURVEY,TAIPEI,TAIWAN.;",TAIWAN,TAIWAN,TAIWAN,4,...,10.3406/bulmi.1988.8093,Article,NaN,MASSON EDITEUR,BULLETIN DE MINERALOGIE,4,111,ISI,NaN,NATL SUN YAT SEN UNIV
1324,1996.0,NaN,Comparing the IRI prediction with the observed...,Cheng K;Huang Y,We compare the equatorial anomaly in the Total...,"MOTC, DIRECTORATE GEN TELECOMMUN, TAIPEI 106, ...",TAIWAN,TAIWAN,TAIWAN,2,...,10.1016/0273-1177(95)00933-7,Article; Proceedings Paper,NaN,PERGAMON PRESS LTD,LOW AND EQUATORIAL LATITUDES IN THE INTERNATIO...,5,18,ISI,NaN,TELECOMMUN TRAINING INST
1326,2001.0,NaN,Ionospheric foF2 variations prior to strong ea...,Chuo Y;Chen Y;Liu J;Pulinets S,Many studies of the seismo-ionospheric couplin...,"Natl Cent Univ, Inst Space Sci, Chungli 320, T...",TAIWAN;TAIWAN;TAIWAN;RUSSIA,TAIWAN,TAIWAN,4,...,10.1016/S0273-1177(01)00209-5,Article; Proceedings Paper,CANNOT EARTHQUAKES; LIGHTS; PREDICTION,ELSEVIER SCIENCE BV,ADVANCES IN REMOTE SENSING OF THE MIDDLE AND U...,19,27,ISI,NATL CENT UNIV;NATL CENT UNIV;NATL CENT UNIV;I...,NATL CENT UNIV
1329,2004.0,NaN,Taiwans cooperative space activities at presen...,Ip W,Taiwan is developing a long-term space program...,"Natl Cent Univ, Inst Astron & Space Sci, Chung...",TAIWAN,TAIWAN,TAIWAN,1,...,10.1016/j.asr.2003.04.055,Article; Proceedings Paper,NaN,PERGAMON-ELSEVIER SCIENCE LTD,SPACE SCIENCE EDUCATION AND PROMOTING NORTH-SO...,0,34,ISI,NATL CENT UNIV,NATL CENT UNIV


In [40]:
for idx, row in df.iterrows():
    doi_str = row['DI']
    api_url = 'https://api.crossref.org/works/' + doi_str
    try: 
        with urlopen(api_url) as url:
            word_json = json.loads(url.read().decode())
        if word_json['message']['title']:
            df.loc[idx, 'Title_verified'] = word_json['message']['title'][0]
        else:
            df.loc[idx, 'Title_verified'] = np.nan
    except HTTPError:
        df.loc[idx, 'Title_verified'] = np.nan

# tmp = df.loc[1, 'DI']
# tmp

/home/jovyan/.conda/envs/jme/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [42]:
df = df.dropna(subset=['Title_verified'])
df

,PY,JI,TI,AU,AB,C1,AU_CO,AU1_CO,AUc_CO,N_AU,...,DT,ID,PU,SO,TC,VL,DB,AU_UN,AUc_UN,Title_verified
1,2011.0,AAPG Mem.,Two contrasting kinematic styles of active fol...,Yue L;Suppe J;Hung J,Two adjacent active thrust ramps in western Ta...,"Department of Geosciences, Princeton Universit...",USA;TAIWAN;TAIWAN;TAIWAN,USA,USA,3,...,Article,coseismic process; deformation; detachment f...,NaN,AAPG Memoir,18,NaN,scopus,PRINCETON UNIVERSITY;NATIONAL TAIWAN UNIVERSIT...,DEPARTMENT OF GEOSCIENCES,Two Contrasting Kinematic Styles of Active Fol...
2,1977.0,Agricultural Administration,ChineseAmerican Joint Commission on Rural Reco...,Wang Y,The Chinese-American Joint Commission on Rural...,"Secretary General, Joint Commission on Rural R...",TAIWAN,TAIWAN,TAIWAN,1,...,Article,NaN,NaN,Agricultural Administration,1,4,scopus,NaN,NOTREPORTED,Chinese-American Joint Commission on Rural Rec...
4,2010.0,Am. J. Sci.,ACCRETIONARY OROGEN AND EVOLUTION OF THE JAPAN...,Jahn B,The Japanese Islands represent a segment of a ...,"Acad Sinica, Inst Earth Sci, Taipei 11529, Tai...",TAIWAN,TAIWAN,TAIWAN,1,...,Article,A-TYPE GRANITES; ARABIAN-NUBIAN SHIELD; PALEOZ...,AMER JOURNAL SCIENCE,AMERICAN JOURNAL OF SCIENCE,72,310,ISI,INST EARTH SCI,NATL TAIWAN UNIV,Accretionary orogen and evolution of the Japan...
5,2021.0,Am. J. Sci.,American Journal of Science,Chang Q;Hren M;Lin A;Tabor C;Yu S;Eley Y;Harris G,Fluvial sediments are important archives of pa...,"Univ Connecticut, Dept Geosci, Storrs, CT 0626...",USA;TAIWAN;TAIWAN;UNITED KINGDOM,USA,USA,7,...,Article,CARBON-ISOTOPE DISCRIMINATION; LAST GLACIAL MA...,AMER JOURNAL SCIENCE,AMERICAN JOURNAL OF SCIENCE,1,321,ISI,UNIV CONNECTICUT;NATL CENT UNIV;EXPLORAT AND D...,UNIV CONNECTICUT,Terrestrial biomarker isotope records of late ...
6,2006.0,Anatolia,Assessing a river tracing behavioural model A ...,Lee T,NaN,Graduate School of Leisure and Exercise Studie...,TAIWAN,TAIWAN,TAIWAN,1,...,Article,model; river; tourism; tourist behavior; t...,Anatolia,Anatolia,9,17,scopus,NATIONAL YUNLIN UNIVERSITY OF SCIENCE AND TECH...,電子郵件: THLEE@YUNTECH.EDU.TW,Assessing a River Tracing Behavioural Model: a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321,1988.0,NaN,MICROSTRUCTURE OF CHROMIUM SPINEL AND PYRITE A...,HWANG S;SHEN P;CHU H;JENG R,NaN,"CENT GEOL SURVEY,TAIPEI,TAIWAN.;",TAIWAN,TAIWAN,TAIWAN,4,...,Article,NaN,MASSON EDITEUR,BULLETIN DE MINERALOGIE,4,111,ISI,NaN,NATL SUN YAT SEN UNIV,Microstructure of chromium spinel and pyrite a...
1324,1996.0,NaN,Comparing the IRI prediction with the observed...,Cheng K;Huang Y,We compare the equatorial anomaly in the Total...,"MOTC, DIRECTORATE GEN TELECOMMUN, TAIPEI 106, ...",TAIWAN,TAIWAN,TAIWAN,2,...,Article; Proceedings Paper,NaN,PERGAMON PRESS LTD,LOW AND EQUATORIAL LATITUDES IN THE INTERNATIO...,5,18,ISI,NaN,TELECOMMUN TRAINING INST,Comparing the IRI prediction with the observed...
1326,2001.0,NaN,Ionospheric foF2 variations prior to strong ea...,Chuo Y;Chen Y;Liu J;Pulinets S,Many studies of the seismo-ionospheric couplin...,"Natl Cent Univ, Inst Space Sci, Chungli 320, T...",TAIWAN;TAIWAN;TAIWAN;RUSSIA,TAIWAN,TAIWAN,4,...,Article; Proceedings Paper,CANNOT EARTHQUAKES; LIGHTS; PREDICTION,ELSEVIER SCIENCE BV,ADVANCES IN REMOTE SENSING OF THE MIDDLE AND U...,19,27,ISI,NATL CENT UNIV;NATL CENT UNIV;NATL CENT UNIV;I...,NATL CENT UNIV,Ionospheric foF2 variations prior to strong ea...
1329,2004.0,NaN,Taiwans cooperative space activities at presen...,Ip W,Taiwan is developing a long-term space program...,"Natl Cent Univ, Inst Astron & Space Sci, Chung...",TAIWAN,TAIWAN,TAIWAN,1,...,Article; Proceedings Paper,NaN,PERGAMON-ELSEVIER SCIENCE LTD,SPACE SCIENCE EDUCATION AND PROMOTING NORTH-SO...,0,34,ISI,NATL CENT UNIV,NATL CENT UNIV,Taiwan’s cooperative space activities at prese...


In [49]:
df.loc[8]

PY                                                           2008.0
JI                                                         Anatolia
TI                 Determining the motivation of wellness travelers
AU                                        Chen J;Prebensen N;Huan T
AB                This research depicts a new investigative them...
C1                Department of Recreation Park and Tourism Stud...
AU_CO                                       INDIA;USA;NORWAY;TAIWAN
AU1_CO                                                        INDIA
AUc_CO                                                          USA
N_AU                                                              3
AU_TW                                                           4.0
FU_TW                                                           NaN
RP                Chen, J.S.; Department of Recreation Park and ...
FU                                                              NaN
FX                                              

In [57]:
cols = df.columns.tolist()
newcols = cols[:3] + [cols[-1]] + cols[3:-1]
df = df[newcols]
df

,PY,JI,TI,Title_verified,AU,AB,C1,AU_CO,AU1_CO,AUc_CO,...,DI,DT,ID,PU,SO,TC,VL,DB,AU_UN,AUc_UN
1,2011.0,AAPG Mem.,Two contrasting kinematic styles of active fol...,Two Contrasting Kinematic Styles of Active Fol...,Yue L;Suppe J;Hung J,Two adjacent active thrust ramps in western Ta...,"Department of Geosciences, Princeton Universit...",USA;TAIWAN;TAIWAN;TAIWAN,USA,USA,...,10.1306/13251337M943431,Article,coseismic process; deformation; detachment f...,NaN,AAPG Memoir,18,NaN,scopus,PRINCETON UNIVERSITY;NATIONAL TAIWAN UNIVERSIT...,DEPARTMENT OF GEOSCIENCES
2,1977.0,Agricultural Administration,ChineseAmerican Joint Commission on Rural Reco...,Chinese-American Joint Commission on Rural Rec...,Wang Y,The Chinese-American Joint Commission on Rural...,"Secretary General, Joint Commission on Rural R...",TAIWAN,TAIWAN,TAIWAN,...,10.1016/0309-586X(77)90003-6,Article,NaN,NaN,Agricultural Administration,1,4,scopus,NaN,NOTREPORTED
4,2010.0,Am. J. Sci.,ACCRETIONARY OROGEN AND EVOLUTION OF THE JAPAN...,Accretionary orogen and evolution of the Japan...,Jahn B,The Japanese Islands represent a segment of a ...,"Acad Sinica, Inst Earth Sci, Taipei 11529, Tai...",TAIWAN,TAIWAN,TAIWAN,...,10.2475/10.2010.02,Article,A-TYPE GRANITES; ARABIAN-NUBIAN SHIELD; PALEOZ...,AMER JOURNAL SCIENCE,AMERICAN JOURNAL OF SCIENCE,72,310,ISI,INST EARTH SCI,NATL TAIWAN UNIV
5,2021.0,Am. J. Sci.,American Journal of Science,Terrestrial biomarker isotope records of late ...,Chang Q;Hren M;Lin A;Tabor C;Yu S;Eley Y;Harris G,Fluvial sediments are important archives of pa...,"Univ Connecticut, Dept Geosci, Storrs, CT 0626...",USA;TAIWAN;TAIWAN;UNITED KINGDOM,USA,USA,...,10.2475/04.2021.01,Article,CARBON-ISOTOPE DISCRIMINATION; LAST GLACIAL MA...,AMER JOURNAL SCIENCE,AMERICAN JOURNAL OF SCIENCE,1,321,ISI,UNIV CONNECTICUT;NATL CENT UNIV;EXPLORAT AND D...,UNIV CONNECTICUT
6,2006.0,Anatolia,Assessing a river tracing behavioural model A ...,Assessing a River Tracing Behavioural Model: a...,Lee T,NaN,Graduate School of Leisure and Exercise Studie...,TAIWAN,TAIWAN,TAIWAN,...,10.1080/13032917.2006.9687194,Article,model; river; tourism; tourist behavior; t...,Anatolia,Anatolia,9,17,scopus,NATIONAL YUNLIN UNIVERSITY OF SCIENCE AND TECH...,電子郵件: THLEE@YUNTECH.EDU.TW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321,1988.0,NaN,MICROSTRUCTURE OF CHROMIUM SPINEL AND PYRITE A...,Microstructure of chromium spinel and pyrite a...,HWANG S;SHEN P;CHU H;JENG R,NaN,"CENT GEOL SURVEY,TAIPEI,TAIWAN.;",TAIWAN,TAIWAN,TAIWAN,...,10.3406/bulmi.1988.8093,Article,NaN,MASSON EDITEUR,BULLETIN DE MINERALOGIE,4,111,ISI,NaN,NATL SUN YAT SEN UNIV
1324,1996.0,NaN,Comparing the IRI prediction with the observed...,Comparing the IRI prediction with the observed...,Cheng K;Huang Y,We compare the equatorial anomaly in the Total...,"MOTC, DIRECTORATE GEN TELECOMMUN, TAIPEI 106, ...",TAIWAN,TAIWAN,TAIWAN,...,10.1016/0273-1177(95)00933-7,Article; Proceedings Paper,NaN,PERGAMON PRESS LTD,LOW AND EQUATORIAL LATITUDES IN THE INTERNATIO...,5,18,ISI,NaN,TELECOMMUN TRAINING INST
1326,2001.0,NaN,Ionospheric foF2 variations prior to strong ea...,Ionospheric foF2 variations prior to strong ea...,Chuo Y;Chen Y;Liu J;Pulinets S,Many studies of the seismo-ionospheric couplin...,"Natl Cent Univ, Inst Space Sci, Chungli 320, T...",TAIWAN;TAIWAN;TAIWAN;RUSSIA,TAIWAN,TAIWAN,...,10.1016/S0273-1177(01)00209-5,Article; Proceedings Paper,CANNOT EARTHQUAKES; LIGHTS; PREDICTION,ELSEVIER SCIENCE BV,ADVANCES IN REMOTE SENSING OF THE MIDDLE AND U...,19,27,ISI,NATL CENT UNIV;NATL CENT UNIV;NATL CENT UNIV;I...,NATL CENT UNIV
1329,2004.0,NaN,Taiwans cooperative space activities at presen...,Taiwan’s cooperative space activities at prese...,Ip W,Taiwan is developing a long-term space program...,"Natl Cent Univ, Inst Astron & Space Sci, Chung...",TAIWAN,TAIWAN,TAIWAN,...,10.1016/j.asr.2003.04.055,Article; Proceedings Paper,NaN,PERGAMON-ELSEVIER SCIENCE LTD,SPACE SCIENCE EDUCATION AND PROMOTING NORTH-SO..

In [58]:
df.to_csv('tentative.csv', index=False)

In [61]:
word_json['message']

{'indexed': {'date-parts': [[2022, 3, 29]],
  'date-time': '2022-03-29T13:38:11Z',
  'timestamp': 1648561091742},
 'reference-count': 24,
 'publisher': 'Elsevier BV',
 'issue': '4',
 'license': [{'start': {'date-parts': [[2006, 1, 1]],
    'date-time': '2006-01-01T00:00:00Z',
    'timestamp': 1136073600000},
   'content-version': 'tdm',
   'delay-in-days': 0,
   'URL': 'https://www.elsevier.com/tdm/userlicense/1.0/'}],
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'short-container-title': ['Advances in Space Research'],
 'published-print': {'date-parts': [[2006, 1]]},
 'DOI': '10.1016/j.asr.2005.08.017',
 'type': 'journal-article',
 'created': {'date-parts': [[2005, 9, 20]],
  'date-time': '2005-09-20T13:03:42Z',
  'timestamp': 1127221422000},
 'page': '747-753',
 'source': 'Crossref',
 'is-referenced-by-count': 8,
 'title': ['A multi-spectral spatial convolution approach of rainfall forecasting using weather satellite imagery'],
 'prefix': '10.1016',
 'volume': '

In [60]:
word_json['message']['title'][0]

'A multi-spectral spatial convolution approach of rainfall forecasting using weather satellite imagery'